In [13]:
import sqlite3
import pandas as pd
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
import torch
import os

# Caminho absoluto do banco de dados
caminho_banco = os.path.abspath('/home/devgir/gir_system/chat/data/base_conhecimento.db')
print(f"Caminho do banco de dados: {caminho_banco}")

# Conectar ao banco de dados
con = sqlite3.connect(caminho_banco)

# Carregar o modelo BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Função para carregar a base de conhecimento
def carregar_base():
    df = pd.read_sql_query("SELECT * FROM conhecimento", con)
    return df

# Função para gerar embeddings com BERT
def gerar_embedding_bert(texto):
    tokens = tokenizer(texto, return_tensors='pt', padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        embeddings = model(**tokens).last_hidden_state.mean(dim=1)
    return embeddings

# Função para encontrar a resposta com base na pergunta do cliente
def encontrar_resposta(pergunta_cliente):
    # Carregar base de conhecimento
    df = carregar_base()

    # Gerar embedding da pergunta do cliente
    embedding_cliente = gerar_embedding_bert(pergunta_cliente)

    # Gerar embeddings das descrições do banco de dados
    embeddings_bd = torch.vstack([gerar_embedding_bert(descricao) for descricao in df['descricao']])

    # Calcular similaridade coseno
    similaridade = cosine_similarity(embedding_cliente.numpy(), embeddings_bd.numpy())

    # Encontrar o índice da descrição mais similar
    indice_max = similaridade.argmax()

    # Retornar a descrição correspondente
    return df.iloc[indice_max]['descricao']

# Testar a IA no notebook
pergunta_cliente = "Como posso alterar minha senha?"
resposta = encontrar_resposta(pergunta_cliente)

print(f"Pergunta: {pergunta_cliente}")
print(f"Resposta: {resposta}")

# Fechar a conexão ao banco de dados ao final
con.close()


Caminho do banco de dados: /home/devgir/gir_system/chat/data/base_conhecimento.db


OperationalError: unable to open database file